In [1]:
import re,time
from selenium import webdriver
driver=webdriver.Chrome()

#获取源代码
def get_html(url):
    driver.get(url)
    time.sleep(5)
    html=driver.page_source
    return html

#获取标号
def get_bh(html):
    list1=[]
    reobj = re.compile(r'<tr><td class="answerIcon_1du7d"></td><td>([\d\D]*?)</td>')
    for match in reobj.finditer(html):
        bh=match.group(1)
        list1.append(bh)
    return list1

#获取网页链接
def get_lj(html):
    list2=[]
    list3=[]
    reobj = re.compile(r'<td><a href="([\d\D]*?)"')
    for match in reobj.finditer(html):
        lj=match.group(1)
        list2.append(lj)
    for i in list2:
        t="https://pintia.cn"+i
        list3.append(t)
    return list3

#获取标题
def get_bt(html):
    list4=[]
    reobj = re.compile(r'<a href="[\d\D]*?" class="">([\d\D]*?)</a>')
    for match in reobj.finditer(html):
        bt=match.group(1)
        list4.append(bt)
    return list4

#获取分数
def get_fs(html):
    list5=[]
    reobj = re.compile(r"</a></td><td>([\d\D]*?)</td>")
    for match in reobj.finditer(html):
        fs=match.group(1)
        list5.append(fs)
    return list5

#获取通过数
def get_tgs(html):
    list6=[]
    reobj = re.compile(r"</a></td><td>[\d\D]*?</td><td>([\d\D]*?)</td>")
    for match in reobj.finditer(html):
        tgl=match.group(1)
        list6.append(tgl)
    return list6

#获取提交数
def get_tjs(html):
    list7=[]
    reobj = re.compile(r"</a></td><td>[\d\D]*?</td><td>[\d\D]*?</td><td>([\d\D]*?)</td>")
    for match in reobj.finditer(html):
        tjs=match.group(1)
        list7.append(tjs)
    return list7

#获取通过率
def get_tgl(html):
    list8=[]
    reobj = re.compile(r"</a></td><td>[\d\D]*?</td><td>[\d\D]*?</td><td>[\d\D]*?</td><td>([\d\D]*?)</td></tr>")
    for match in reobj.finditer(html):
        tgl=match.group(1)
        list8.append(tgl)
    return list8

urls = [("https://pintia.cn/problem-sets/1111652100718116864/problems/type/6","函数题"),("https://pintia.cn/problem-sets/1111652100718116864/problems/type/7","编程题")]
result=[]
for url,category in urls:
    html=get_html(url)
    bh=get_bh(html)
    lj=get_lj(html)
    bt=get_bt(html)
    fs=get_fs(html)
    tgs=get_tgs(html)
    tjs=get_tjs(html)
    tgl=get_tgl(html)
    for a,b,c,d,e,f,g in zip(bh,lj,bt,fs,tgs,tjs,tgl):
        t=(category,a,b,c,d,e,f,g)
        result.append(t)
print("OK")

OK


In [2]:
#获取连接内的数据
def get_sj(html1):
    list9=[]
    reobj = re.compile(r'<div class="problem_Yd8rq p-3 mb-3 background-grey-1"><div class="ques-view">([\d\D]*?)</div><div class="problemInfo_OZ-Rv">')
    for match in reobj.finditer(html1):
        sj=match.group()
        list9.append(sj)
    return list9

final_result=[]
#调用上面的result内的数据，获取其中的链接，打开，获取数据
for i in result:
    url1=i[2]
    driver=webdriver.Chrome()
    driver.get(url1)
    time.sleep(5)
    html1=driver.page_source
    sj=get_sj(html1)         #连接中的内容
    sj1=str(sj)
    z=(i[0],i[1],i[2],i[3],i[4],i[5],i[6],i[7],sj1)   #合并到z中
    final_result.append(z)
    driver.close() #会关闭打开并且获取好的网页，以免打开的太多
print("OK")

OK


In [3]:
from pony.orm import *


db = Database()   #创建数据库对象


class Ttitle(db.Entity):
    lx = Optional(str)
    bh = PrimaryKey(str, auto=True)
    url = Optional(str)
    bt = Optional(str)
    fs = Optional(str)
    tgs = Optional(str)
    tjs = Optional(str)
    tgl = Optional(str)
    sj = Optional(str)



#db.generate_mapping()

In [4]:
db.bind(provider='sqlite', filename='D:/code2020/program.sqlite')

db.generate_mapping(create_tables=True)
#将实体映射到数据库表,参数create_tables = True表示如果实体指向的表尚不存在，则使用CREATE TABLE命令创建它们。必须在调用generate_mapping（）方法之前定义连接到数据库的所有实体。

set_sql_debug(True)
#调试模式,使用set_sql_debug（）函数，您可以看到Pony发送到数据库的SQL命令。

OSError: Database file is not found: 'D:/code2020/program.sqlite'

In [ ]:
for x in final_result:
    s = Ttitle(lx=x[0],bh=x[1],url=x[2],bt=x[3],fs=x[4],tgs=x[5],tjs=x[6],tgl=x[7],sj=x[8])
    db.commit()
    #Pony不会立即在数据库中保存对象。只有在调用commit()函数后才会保存这些对象。如果打开调试模式，那么在commit（）期间，您将看到发送到数据库的五个insert命令。